### Manipulation of PeaskDB de novo-assisted database search results of Trocas 8 (low water, November 2019) non incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/686_Trocas8_June2021_PEAKS_145/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/686_Trocas8_June2021_PEAKS_145


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 686_Trocas8_June2021_PEAKS_145_peptide.csv     'de novo only peptides.csv'
 686_Trocas8_June2021_PEAKS_145_proteins.csv     peptides_1_1_0.mzid
 686_Trocas8_June2021_PEAKS_145_proteins.fasta   protein-peptides.csv
'DB search psm.csv'


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup686 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/686_Trocas8_June2021_PEAKS_145/686_Trocas8_June2021_PEAKS_145_peptide.csv")

# remove redundant rows
peaksdb686 = pd.DataFrame.drop_duplicates(peaksdbdup686)

print(peaksdb686.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb686.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb686['#Spec']
del peaksdb686['Accession']
del peaksdb686['PTM']
del peaksdb686['AScore']

mean_length = peaksdb686['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup686))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb686))

#look at the dataframe
peaksdb686.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Sample 12', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Sample 12', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.08695652173913
# redundant peaksdb peptides in combined dataframe 46
# nonredundant peaksdb peptides in combined dataframe 46


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,GSGGGSSGGSIGGR,55.27,1091.4955,14,-1.9,546.7540,17.65,60100.0,12,3030,20210624_Trocas8_686_Chaves61_surface_DDA_120m...
1,LSSPATLNSR,49.42,1044.5564,10,-2.1,523.2844,44.80,10800000.0,12,8842,20210624_Trocas8_686_Chaves61_surface_DDA_120m...
2,LVNELTEFAK,41.27,1162.6233,10,-2.0,582.3177,86.06,26200.0,12,15683,20210624_Trocas8_686_Chaves61_surface_DDA_120m...
3,LATVISPR,38.42,855.5178,8,0.8,428.7665,59.67,9410000.0,12,11583,20210624_Trocas8_686_Chaves61_surface_DDA_120m...
4,LATVLSPR,38.42,855.5178,8,0.8,428.7665,59.67,9410000.0,12,11583,20210624_Trocas8_686_Chaves61_surface_DDA_120m...


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb686['A'] = peaksdb686['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb686['C'] = peaksdb686['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb686['D'] = peaksdb686['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb686['E'] = peaksdb686['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb686['F'] = peaksdb686['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb686['G'] = peaksdb686['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb686['H'] = peaksdb686['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb686 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb686['I'] = peaksdb686['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb686['K'] = peaksdb686['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb686['L'] = peaksdb686['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb686['M'] = peaksdb686['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb686['N'] = peaksdb686['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb686['P'] = peaksdb686['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb686['Q'] = peaksdb686['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb686['R'] = peaksdb686['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb686['S'] = peaksdb686['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb686['T'] = peaksdb686['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb686['V'] = peaksdb686['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb686['W'] = peaksdb686['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb686['Y'] = peaksdb686['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb686['c-carb'] = peaksdb686['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb686['m-oxid'] = peaksdb686['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb686['n-deam'] = peaksdb686['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb686['q-deam'] = peaksdb686['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb686['stripped_peptide'] = peaksdb686['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb686['stripped_length'] = peaksdb686['stripped_peptide'].apply(len)

peaksdb686['NAAF_num.'] = peaksdb686['Area'] / peaksdb686['stripped_length']

# total the number of modifications in sequence
peaksdb686['ptm-total'] = peaksdb686['c-carb'] + peaksdb686['m-oxid'] + peaksdb686['n-deam'] + peaksdb686['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb686['stripped_IL']= peaksdb686['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb686.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB.csv")

# check out the results
peaksdb686.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,GSGGGSSGGSIGGR,55.27,1091.4955,14,-1.9,546.7540,17.65,60100.0,12,3030,...,0,0,0,0,0,GSGGGSSGGSIGGR,14,4.292857e+03,0,GSGGGSSGGSLGGR
1,LSSPATLNSR,49.42,1044.5564,10,-2.1,523.2844,44.80,10800000.0,12,8842,...,0,0,0,0,0,LSSPATLNSR,10,1.080000e+06,0,LSSPATLNSR
2,LVNELTEFAK,41.27,1162.6233,10,-2.0,582.3177,86.06,26200.0,12,15683,...,0,0,0,0,0,LVNELTEFAK,10,2.620000e+03,0,LVNELTEFAK
3,LATVISPR,38.42,855.5178,8,0.8,428.7665,59.67,9410000.0,12,11583,...,0,0,0,0,0,LATVISPR,8,1.176250e+06,0,LATVLSPR
4,LATVLSPR,38.42,855.5178,8,0.8,428.7665,59.67,9410000.0,12,11583,...,0,0,0,0,0,LATVLSPR,8,1.176250e+06,0,LATVLSPR


In [6]:
# keep only stripped peptide I/L and NAAF
db_686 = peaksdb686[['stripped_IL', 'Area', 'NAAF_num.']]

db_686.set_index('stripped_IL')

# write modified dataframe to new txt file
db_686.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/686_CV_surf_GF75_PDB_ILnaafs.csv")

db_686.head()

,stripped_IL,Area,NAAF_num.
0,GSGGGSSGGSLGGR,60100.0,4.292857e+03
1,LSSPATLNSR,10800000.0,1.080000e+06
2,LVNELTEFAK,26200.0,2.620000e+03
3,LATVLSPR,9410000.0,1.176250e+06
4,LATVLSPR,9410000.0,1.176250e+06


### Exporting txt files of stripped peptides at confidence cutoffs:

In [8]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep686moddup = peaksdb686[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep686dup = peaksdb686[["stripped_peptide"]]

# deduplicate both of these lists
pep686mod = pep686moddup.drop_duplicates()
pep686 = pep686dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 686:", len(pep686moddup))
print("Deduplicated modified PeaksDB peptides in 686:", len(pep686mod))
print("Total stripped PeaksDB peptides in 686:", len(pep686dup))
print("Deduplicated stripped PeaksDB peptides in 686:", len(pep686))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep686.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB_stripped_peptides.fas

# look at the stripped peptides
pep686.head()

Total modified PeaksDB peptides in 686: 46
Deduplicated modified PeaksDB peptides in 686: 46
Total stripped PeaksDB peptides in 686: 46
Deduplicated stripped PeaksDB peptides in 686: 46


,stripped_peptide
0,GSGGGSSGGSIGGR
1,LSSPATLNSR
2,LVNELTEFAK
3,LATVISPR
4,LATVLSPR


## NAAF correction and exporting files with AA and PTM totals:

In [9]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb686['A'].sum(),
        'C': peaksdb686['C'].sum(),
        'D': peaksdb686['D'].sum(),
        'E': peaksdb686['E'].sum(),
        'F': peaksdb686['F'].sum(),
        'G': peaksdb686['G'].sum(),
        'H': peaksdb686['H'].sum(),
        'I': peaksdb686['I'].sum(),
        'K': peaksdb686['K'].sum(),
        'L': peaksdb686['L'].sum(),
        'M': peaksdb686['M'].sum(),
        'N': peaksdb686['N'].sum(),
        'P': peaksdb686['P'].sum(),
        'Q': peaksdb686['Q'].sum(),
        'R': peaksdb686['R'].sum(),
        'S': peaksdb686['S'].sum(),
        'T': peaksdb686['T'].sum(),
        'V': peaksdb686['V'].sum(),
        'W': peaksdb686['W'].sum(),
        'Y': peaksdb686['Y'].sum(),
        'c-carb': peaksdb686['c-carb'].sum(),
        'm-oxid': peaksdb686['m-oxid'].sum(),
        'n-deam': peaksdb686['n-deam'].sum(),
        'q-deam': peaksdb686['q-deam'].sum(),
        'Total area': peaksdb686['Area'].sum(),
        'Total length': peaksdb686['stripped_length'].sum()
       }

totalpeaksdb686 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb686['% C w/ carb'] = totalpeaksdb686['c-carb'] / totalpeaksdb686['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb686['% M w/ oxid'] = totalpeaksdb686['m-oxid'] / totalpeaksdb686['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb686['% N w/ deam'] = totalpeaksdb686['n-deam'] / totalpeaksdb686['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb686['% Q w/ deam'] = totalpeaksdb686['q-deam'] / totalpeaksdb686['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb686['NAAF denom.'] = totalpeaksdb686['Total area'] / totalpeaksdb686['Total length']

# write modified dataframe to new txt file
totalpeaksdb686.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB_totals.csv")

totalpeaksdb686.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,41,2,14,16,9,40,11,33,31,55,...,4,3,2,66944302.0,425,1.0,0.666667,0.157895,0.25,157516.004706


In [10]:
# use the calculated NAAF factor (in totalpeaksdb686 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 157516.004706

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb686['NAAF factor'] = (peaksdb686['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksdb686_NAAF = peaksdb686[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor CV the AAs to normalize its abundance CV peak area and peptide length

peaksdb686_NAAF['A-NAAF20'] = peaksdb686_NAAF['A'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['C-NAAF20'] = peaksdb686_NAAF['C'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['D-NAAF20'] = peaksdb686_NAAF['D'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['E-NAAF20'] = peaksdb686_NAAF['E'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['F-NAAF20'] = peaksdb686_NAAF['F'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['G-NAAF20'] = peaksdb686_NAAF['G'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['H-NAAF20'] = peaksdb686_NAAF['H'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['I-NAAF20'] = peaksdb686_NAAF['I'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['K-NAAF20'] = peaksdb686_NAAF['K'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['L-NAAF20'] = peaksdb686_NAAF['L'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['M-NAAF20'] = peaksdb686_NAAF['M'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['N-NAAF20'] = peaksdb686_NAAF['N'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['P-NAAF20'] = peaksdb686_NAAF['P'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['Q-NAAF20'] = peaksdb686_NAAF['Q'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['R-NAAF20'] = peaksdb686_NAAF['R'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['S-NAAF20'] = peaksdb686_NAAF['S'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['T-NAAF20'] = peaksdb686_NAAF['T'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['V-NAAF20'] = peaksdb686_NAAF['V'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['W-NAAF20'] = peaksdb686_NAAF['W'] * peaksdb686['NAAF factor']
peaksdb686_NAAF['Y-NAAF20'] = peaksdb686_NAAF['Y'] * peaksdb686['NAAF factor']

# multiply the NAAF20 factor CV the PTMs normalize its abundance CV peak area and peptide length

peaksdb686_NAAF['ccarb-NAAF20'] = peaksdb686_NAAF['c-carb'] * peaksdb686_NAAF['NAAF factor']
peaksdb686_NAAF['moxid-NAAF20'] = peaksdb686_NAAF['m-oxid'] * peaksdb686_NAAF['NAAF factor']
peaksdb686_NAAF['ndeam-NAAF20'] = peaksdb686_NAAF['n-deam'] * peaksdb686_NAAF['NAAF factor']
peaksdb686_NAAF['qdeam-NAAF20'] = peaksdb686_NAAF['q-deam'] * peaksdb686_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb686_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB_naaf.csv")

peaksdb686_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,GSGGGSSGGSIGGR,0.027253,0,0,0,0,0,8,0,1,...,0.027253,0.109014,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,LSSPATLNSR,6.856446,1,0,0,0,0,0,0,0,...,6.856446,20.569338,6.856446,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,LVNELTEFAK,0.016633,1,0,0,2,1,0,0,0,...,0.000000,0.000000,0.016633,0.016633,0.0,0.0,0.0,0.0,0.0,0.0
3,LATVISPR,7.467495,1,0,0,0,0,0,0,1,...,7.467495,7.467495,7.467495,7.467495,0.0,0.0,0.0,0.0,0.0,0.0
4,LATVLSPR,7.467495,1,0,0,0,0,0,0,0,...,7.467495,7.467495,7.467495,7.467495,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# made a new dataframe that contains the sums of NAAF normalized AAs for peaksdb686 peaksdb results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb686_NAAF['NAAF factor'].sum(),
        'A': peaksdb686_NAAF['A-NAAF20'].sum(),
        'C': peaksdb686_NAAF['C-NAAF20'].sum(),
        'D': peaksdb686_NAAF['D-NAAF20'].sum(),
        'E': peaksdb686_NAAF['E-NAAF20'].sum(),
        'F': peaksdb686_NAAF['F-NAAF20'].sum(),
        'G': peaksdb686_NAAF['G-NAAF20'].sum(),
        'H': peaksdb686_NAAF['H-NAAF20'].sum(),
        'I': peaksdb686_NAAF['I-NAAF20'].sum(),
        'K': peaksdb686_NAAF['K-NAAF20'].sum(),
        'L': peaksdb686_NAAF['L-NAAF20'].sum(),
        'M': peaksdb686_NAAF['M-NAAF20'].sum(),
        'N': peaksdb686_NAAF['N-NAAF20'].sum(),
        'P': peaksdb686_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb686_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb686_NAAF['R-NAAF20'].sum(),
        'S': peaksdb686_NAAF['S-NAAF20'].sum(),
        'T': peaksdb686_NAAF['T-NAAF20'].sum(),
        'V': peaksdb686_NAAF['V-NAAF20'].sum(),
        'W': peaksdb686_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb686_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb686_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb686_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb686_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb686_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb686_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb686_NAAF['% C w/ carb.'] = totalpeaksdb686_NAAF['c-carb'] / totalpeaksdb686_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb686_NAAF['% M w/ oxid'] = totalpeaksdb686_NAAF['m-oxid'] / totalpeaksdb686_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb686_NAAF['% N w/ deam'] = totalpeaksdb686_NAAF['n-deam'] / totalpeaksdb686_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb686_NAAF['% Q w/ deam'] = totalpeaksdb686_NAAF['q-deam'] / totalpeaksdb686_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb686_NAAF['NAAF check'] = totalpeaksdb686_NAAF['NAAF'] / 157516.004706

# write modified dataframe to new txt file, same name + totals
totalpeaksdb686_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDB/678_CV_surf_GF75_PDB/686_CV_surf_GF75_PDB_naaf_totals.csv")

totalpeaksdb686_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,51.572489,37.462528,0.00386,0.418658,0.310646,0.220379,1.329711,0.668461,19.086684,1.011427,...,0.098205,0.00386,0.017937,0.014389,0.015089,1.0,0.092161,0.001979,0.296357,0.000327
